In [ ]:
%load_ext autoreload
%autoreload 2

import wandb

In [ ]:
seg_runs = [
    ('8bh5rurv', 0.8848695158958435),
    ('cnqyhmsb', 0.8836771249771118),
    ('ri1dvvam', 0.8832937479019165),
    ('ilhz9vkq', 0.883250892162323),
    ('dhmhz041', 0.8829216957092285),
    ('7mrir9a3', 0.8828873634338379),
    ('yzmxfvrv', 0.8828799724578857),
    ('5ywrt6qn', 0.882686972618103),
    ('ighbwtdm', 0.8826081156730652),
    ('315hd0sm', 0.8825200796127319),
    ('y2jgqdox', 0.8821526169776917),
    ('xr44njvv', 0.8815780282020569),
    ('mb22v5ps', 0.8812705874443054),
    ('q6d1n08d', 0.8809899091720581),
    ('37y0hk18', 0.8808367848396301),
    ('3ag6fjep', 0.8806321024894714),
    ('uevn0e5g', 0.8803757429122925),
    ('0en8eomx', 0.8799382448196411),
    ('o0wao11j', 0.8796645998954773),
    ('4qrallkn', 0.8785540461540222),
    ('6vya7uxd', 0.8780744671821594),
    ('rwa0acur', 0.8775778412818909),
    ('qoijsx0h', 0.8775392770767212),
    ('l1hf5uuh', 0.8767494559288025),
    ('ebwyd2di', 0.8765263557434082),
    ('6g0nkv0j', 0.8760395646095276),
    ('ph4c64ar', 0.8760202527046204),
    ('amkk3cbw', 0.8735491037368774),
    ('c1dlbdub', 0.8734628558158875),
    ('hiinbqsu', 0.8719046115875244)
]

seg_dam_runs = [
    ('tyi7q17m', 0.8150793196256518),
    ('ngmxec6s', 0.8096530334019485),
    ('vrgbzaup', 0.8096383585083926),
    ('443t9312', 0.8083381310832123),
    ('pxcuoulb', 0.8081565006839244),
    ('cpumt46r', 0.8080391959243641),
    ('viptnxip', 0.8076960005628842),
    ('mhczmsoa', 0.8073159432491346),
    ('znxzcx0a', 0.8072744354969165),
    ('7ar6am07', 0.8071092834046755),
    ('lwe6n5c0', 0.8069356023059625),
    ('90mnu920', 0.8069068799555296),
    ('08meqrfc', 0.806507592996532),
    ('j4xmz3we', 0.8061300715307924),
    ('5ocdy37k', 0.806018066403344),
    ('5pn3bp2o', 0.8057287190805172),
    ('0gvvydkt', 0.8057094304994589),
    ('or34wjnl', 0.8056978455958853),
    ('8n17nhye', 0.8053004703929182),
    ('c2hvohg7', 0.8050853487741387),
    ('33aasxzr', 0.8048996430920557),
    ('40l1n8y6', 0.804133361539285),
    ('8slg69rb', 0.8039897395816178),
    ('yzy5xz0v', 0.8037504662087867),
    ('71urz3vc', 0.803504510967261),
    ('plef5k0s', 0.8033634254385499),
    ('rfu2ifuh', 0.8031063433726832),
    ('5zu1m3bn', 0.8028573049710269),
    ('nj4eqar6', 0.8022980617768509),
    ('95pmh37c', 0.8022721609563958)
]

dam_od_runs = [
    ('kx9yzcb9', 0.8188149291818385),
    ('wbqskbmu', 0.8104202275388073),
    ('y7n4yzya', 0.808878301295482),
    ('c1lit0gf', 0.8086308781844662),
    ('ygnsmpis', 0.8047019871807307),
    ('t91jg9hv', 0.8030884276616155),
    ('5v3elkfr', 0.8026197042480495),
    ('0qjy2h9e', 0.8011403695871695),
    ('6s629jji', 0.7983212991363049),
    ('umj3wmwh', 0.7972560052280561),
    ('phtv4ekc', 0.7952364835570059),
    ('ypydqpw5', 0.7950183345724261),
    ('9bv7h3cw', 0.7941133157892892),
    ('w8ka9fat', 0.7932566433728835),
    ('zu0uhkyk', 0.7921109978206695)
]

In [ ]:
TOP_SEGMENT = 10
TOP_DAM_SEG = 10
TOP_DAM_OD = 5

seg_ids = [r_id for r_id, _ in seg_runs[:TOP_SEGMENT]]
seg_dam_ids = [r_id for r_id, _ in seg_dam_runs[:TOP_DAM_SEG]]
dam_od_ids = [r_id for r_id, _ in dam_od_runs[:TOP_DAM_OD]]

## Building Ensemble

In [ ]:
import numpy as np

In [ ]:
y = np.load('../dev/buildings.npy')

X = [np.load(f'../dev/segment/{run_id}.npy') for run_id in seg_ids]
X = np.concatenate(X, axis=1)

n, nf, *_ = X.shape

X = X.reshape(n, nf, -1)
X = X.transpose(0,2,1).reshape(-1, nf)

y = y.reshape(n, -1)
y = y.reshape(X.shape[0])

In [ ]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_predict

In [ ]:
from lightgbm.sklearn import LGBMClassifier
model = LGBMClassifier(n_jobs=-1)

In [ ]:
n = X.shape[0]
N_SPLITS = 5
split_size = n//N_SPLITS

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
kf = KFold(3)
preds = []

for train_index, test_index in tqdm(kf.split(X,y), total=3):
    model.fit(X[train_index], y[train_index])
    preds.append(model.predict_proba(X[test_index])[:,1])

In [ ]:
#preds = cross_val_predict(model, X, y, method="predict_proba", verbose=100)[:,1]

In [ ]:
probs = model.predict_proba(X)[:,1]

In [ ]:
y_pred = preds > 0.5

In [ ]:
f1_score(y, y_pred)

In [ ]:
# best model: 0.8842283147668784
# mean ensemble: 0.8872299181077064
# model ensemble: 0.8821699586706273

In [ ]:
from xv import io
import ttach as tta
import torch
from tqdm import tqdm_notebook as tqdm
import numpy as np
from xv.tta import BoxClassifierTTA

@torch.no_grad()
def predict_seg_run(run_id, project, to):
    seg_run_path = f"xvr-hlt/{project}/{run_id}"
    conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name
    conf = io.Config(conf_file)
    conf.filter_none = False # predict on all
    
    model, preproc = io.load_segmentation_model(conf, state_file)
    model.eval().cuda();
    model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
    
    dev_data, dev_loader = io.load_dev_data(conf, preproc)
    
    ars = []
    
    for im, mask in tqdm(dev_loader):
        out = model(im.cuda()).sigmoid()
        ars.append(out.cpu().numpy())
    
    preds = np.concatenate(ars)
    np.save(f'{to}/{run_id}', preds)

@torch.no_grad()
def predict_od(run_id, project, to):
    seg_run_path = f"xvr-hlt/{project}/{run_id}"
    conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name
    conf = io.Config(conf_file)

    dmg_model = io.load_damage_model(conf, state_file)
    dmg_model = BoxClassifierTTA(dmg_model)
    dmg_model = dmg_model.eval().cuda()
    return dmg_model

In [ ]:
for run_id in tqdm(seg_ids[10:]):
    predict_seg_run(run_id, 'sky-eye-full', '../dev/segment/')

In [ ]:
for run_id in tqdm(seg_dam_ids[10:20]):
    predict_seg_run(run_id, 'building-seg-damage', '../dev/segment-damage/')

In [ ]:
import wandb

api = wandb.Api()

PROJECT = 'sky-eye-full'
METRIC = "building:f1"

#PROJECT = 'building-seg-damage'
#METRIC = "hmean:damage:categorical:f1"

#PROJECT = 'building-damage'
#METRIC = "hmean:damage:categorical:f1"


runs = api.runs(f'xvr-hlt/{PROJECT}')

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm_notebook as tqdm

with ThreadPoolExecutor(16) as pool:
    scores = list(tqdm(pool.map(lambda r: get_score(r, metric=METRIC), runs)))
    
scores = dict(zip(runs, scores))

kvs = []
vs = set()

for k,v in sorted(((k,v) for k,v in scores.items() if v is not None), key=lambda kv: -kv[1]):
    if v in vs:
        continue
    vs.add(v)
    kvs.append((k,v))